In [1]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm 

# Don't collapse Pandas Dataframes:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
def normalize(data): ## Normalize Data between 0 and 1
    min_val = min(data)
    if min_val < 0:
        data = [x + abs(min_val) for x in data]
    max_val = max(data)
    normalized_data = [x/max_val for x in data]
    return normalized_data
  
def one_hot_encoding_column(df, column):
    ohe_df = pd.get_dummies(df[column])
    ohe_df.reset_index(drop = True, inplace = True)
    df = pd.concat([df, ohe_df], axis = 1)
    return df

class CBRecommend():
    def __init__(self, df):
        self.df = df
        
    def cosine_sim(self, v1,v2):
        return sum(dot(v1,v2)/(norm(v1)*norm(v2)))
    
    def recommend(self, book_id, n_rec):
        # calculate similarity of input book_id vector w.r.t all other vectors
        inputVec = self.df.loc[book_id].values
        self.df['Cosine Similarity']= self.df.apply(lambda x: self.cosine_sim(inputVec, x.values), axis=1)

        # returns top n user specified books
        return self.df.nlargest(columns='Cosine Similarity',n=n_rec)

# def cosine_sim(v1,v2):
#     cosine_similarity = sum(dot(v1,v2)/(norm(v1)*norm(v2)))
#     return cosine_similarity

# def recommend(df, book_id, number_of_recommendations):
#     inputVec = df.loc[book_id].values
#     df['Cosine Similarity']= df.apply(lambda x: cosine_sim(inputVec, x.values), axis=1)
#     df = df.nlargest(columns='Cosine Similarity', n=number_of_recommendations) # returns top n user specified books
#     return df


In [6]:
## Import Dataset
df = pd.read_csv('dataset.csv')
df['num_pages_norm'] = normalize(df['num_pages'].values)
df['book_rating_norm'] = normalize(df['book_rating'].values)
df['book_price_norm'] = normalize(df['book_price'].values)
print('Dataset Shape: {0}'.format(df.shape))
df.sort_values(['book_id'], ascending=False).head(20)

Dataset Shape: (100000, 13)


,book_id,author_id,book_genre,user_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang,num_pages_norm,book_rating_norm,book_price_norm
85340,3000,139,1,29561,479,8,33,2016,92,7,0.684286,0.8,0.460
22544,3000,213,5,1954,159,5,42,2004,159,1,0.227143,0.5,0.795
63678,3000,402,1,4738,377,4,50,2004,26,6,0.538571,0.4,0.130
65210,3000,7,7,1794,145,5,2,2014,141,4,0.207143,0.5,0.705
94237,3000,313,5,10960,350,6,48,2010,78,2,0.500000,0.6,0.390
43063,3000,187,5,12893,664,6,2,2014,72,1,0.948571,0.6,0.360
77475,3000,39,3,12174,691,2,16,2020,81,5,0.987143,0.2,0.405
68483,3000,376,3,1928,329,8,27,2001,85,4,0.470000,0.8,0.425
25087,3000,282,3,1627,87,10,40,2003,79,5,0.124286,1.0,0.395
58198,3000,69,2,16670,123,7,45,2002,163,7,0.175714,0.7,0.815


In [7]:
df = one_hot_encoding_column(df = df, column = 'book_genre') # Feature & Labels
df = one_hot_encoding_column(df = df, column = 'publisher_id') # Feature & Labels
df = one_hot_encoding_column(df = df, column = 'book_rating_norm') # Feature & Labels
df.drop(columns = ['publish_year', 'num_pages', 'book_rating', 'book_price', 'text_lang'], inplace = True) # Remove Columns that aren't Features
df.set_index('book_id', inplace = True) # Set Instance as Index
df.sort_values(['book_id'], ascending=False).head()

,author_id,book_genre,user_id,publisher_id,num_pages_norm,book_rating_norm,book_price_norm,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
book_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3000,139,1,29561,33,0.684286,0.8,0.460,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3000,213,5,1954,42,0.227143,0.5,0.795,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3000,402,1,4738,50,0.538571,0.4,0.130,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3000,7,7,1794,2,0.207143,0.5,0.705,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3000,313,5,10960,48,0.500000,0.6,0.390,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0


In [8]:
# ran on a sample as an example
book_id = 1001
number_of_recommendations = 20
# recommendation_df = recommend(df, book_id, number_of_recommendations)
cbr = CBRecommend(df)
recommendation_df = cbr.recommend(book_id, number_of_recommendations)
recommendation_df = recommendation_df[['book_genre','publisher_id','book_rating_norm','Cosine Similarity']]
recommendation_df # As the cosine similarity measurement gets closer to 1, then the angle between the two vectors A and B is smaller

,book_genre,publisher_id,book_rating_norm,Cosine Similarity
book_id,,,,
2259,8,38,0.2,4.792067
2737,7,31,0.8,4.792067
924,9,45,0.9,4.792067
2351,10,42,0.3,4.792067
602,7,30,1.0,4.792067
2119,10,43,0.2,4.792067
184,6,31,0.4,4.792067
1617,7,32,1.0,4.792067
1097,9,42,0.2,4.792067


In [ ]:
# ran on a sample as an example
book_id = 1318
number_of_recommendations = 20
recommendation_df = recommend(df, book_id, number_of_recommendations)
recommendation_df = recommendation_df[['author_id','publisher_id','book_rating_norm','Cosine Similarity']]
recommendation_df # As the cosine similarity measurement gets closer to 1, then the angle between the two vectors A and B is smaller

In [ ]:
# ran on a sample as an example
book_id = 700
number_of_recommendations = 20
recommendation_df = recommend(df, book_id, number_of_recommendations)
recommendation_df = recommendation_df[['author_id','publisher_id','book_rating_norm','Cosine Similarity']]
recommendation_df # As the cosine similarity measurement gets closer to 1, then the angle between the two vectors A and B is smaller

In [ ]:
# ran on a sample as an example
book_id = 302
number_of_recommendations = 20
recommendation_df = recommend(df, book_id, number_of_recommendations)
recommendation_df = recommendation_df[['author_id','publisher_id','book_rating_norm','Cosine Similarity']]
recommendation_df # As the cosine similarity measurement gets closer to 1, then the angle between the two vectors A and B is smaller